In [ ]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.model_zoo as model_zoo

BatchNorm2d = nn.BatchNorm2d

class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, dilation=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               dilation=dilation, padding=dilation, bias=False)
        self.bn2 = BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride
        self.dilation = dilation

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

class ResNet(nn.Module):

    def __init__(self, nInputChannels, block, layers, os=16, pretrained=False):
        self.inplanes = 64
        super(ResNet, self).__init__()
        if os == 16:
            strides = [1, 2, 2, 1]
            dilations = [1, 1, 1, 2]
            blocks = [1, 2, 4]
        elif os == 8:
            strides = [1, 2, 1, 1]
            dilations = [1, 1, 2, 2]
            blocks = [1, 2, 1]
        else:
            raise NotImplementedError

        # Modules
        self.conv1 = nn.Conv2d(nInputChannels, 64, kernel_size=7, stride=2, padding=3,
                                bias=False)
        self.bn1 = BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(block, 64, layers[0], stride=strides[0], dilation=dilations[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=strides[1], dilation=dilations[1])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=strides[2], dilation=dilations[2])
        self.layer4 = self._make_MG_unit(block, 512, blocks=blocks, stride=strides[3], dilation=dilations[3])

        self._init_weight()

        if pretrained:
            self._load_pretrained_model()

    def _make_layer(self, block, planes, blocks, stride=1, dilation=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, dilation, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def _make_MG_unit(self, block, planes, blocks=[1, 2, 4], stride=1, dilation=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, dilation=blocks[0]*dilation, downsample=downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, len(blocks)):
            layers.append(block(self.inplanes, planes, stride=1, dilation=blocks[i]*dilation))

        return nn.Sequential(*layers)

    def forward(self, input):
        x = self.conv1(input)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        low_level_feat = x
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        return x, low_level_feat

    def _init_weight(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _load_pretrained_model(self):
        pretrain_dict = torch.load('/personal/预训练模型/resnet101.pth')
        model_dict = {}
        state_dict = self.state_dict()
        for k, v in pretrain_dict.items():
            if k in state_dict and v.shape == state_dict[k].shape:
                model_dict[k] = v
        state_dict.update(model_dict)
        self.load_state_dict(state_dict)

def ResNet101(nInputChannels=3, os=16, pretrained=False):
    model = ResNet(nInputChannels, Bottleneck, [3, 4, 23, 3], os, pretrained=pretrained)
    return model


class ASPP_module(nn.Module):
    def __init__(self, inplanes, planes, dilation):
        super(ASPP_module, self).__init__()
        if dilation == 1:
            kernel_size = 1
            padding = 0
        else:
            kernel_size = 3
            padding = dilation
        self.atrous_convolution = nn.Conv2d(inplanes, planes, kernel_size=kernel_size,
                                            stride=1, padding=padding, dilation=dilation, bias=False)
        self.bn = BatchNorm2d(planes)
        self.relu = nn.ReLU()

        self._init_weight()

    def forward(self, x):
        x = self.atrous_convolution(x)
        x = self.bn(x)

        return self.relu(x)

    def _init_weight(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()


class DeepLabv3_plus(nn.Module):
    def __init__(self, nInputChannels=3, n_classes=21, os=16, pretrained=False, freeze_bn=False, _print=True):
        if _print:
            print("Constructing DeepLabv3+ model...")
            print("Backbone: Resnet-101")
            print("Number of classes: {}".format(n_classes))
            print("Output stride: {}".format(os))
            print("Number of Input Channels: {}".format(nInputChannels))
        super(DeepLabv3_plus, self).__init__()

        # Atrous Conv
        self.resnet_features = ResNet101(nInputChannels, os, pretrained=pretrained)

        # ASPP
        if os == 16:
            dilations = [1, 6, 12, 18]
        elif os == 8:
            dilations = [1, 12, 24, 36]
        else:
            raise NotImplementedError

        self.aspp1 = ASPP_module(2048, 256, dilation=dilations[0])
        self.aspp2 = ASPP_module(2048, 256, dilation=dilations[1])
        self.aspp3 = ASPP_module(2048, 256, dilation=dilations[2])
        self.aspp4 = ASPP_module(2048, 256, dilation=dilations[3])

        self.relu = nn.ReLU()

        self.global_avg_pool = nn.Sequential(nn.AdaptiveAvgPool2d((1, 1)),
                                             nn.Conv2d(2048, 256, 1, stride=1, bias=False),
                                             BatchNorm2d(256),
                                             nn.ReLU())

        self.conv1 = nn.Conv2d(1280, 256, 1, bias=False)
        self.bn1 = BatchNorm2d(256)

        # adopt [1x1, 48] for channel reduction.
        self.conv2 = nn.Conv2d(256, 48, 1, bias=False)
        self.bn2 = BatchNorm2d(48)

        self.last_conv = nn.Sequential(nn.Conv2d(304, 256, kernel_size=3, stride=1, padding=1, bias=False),
                                       BatchNorm2d(256),
                                       nn.ReLU(),
                                       nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=False),
                                       BatchNorm2d(256),
                                       nn.ReLU(),
                                       nn.Conv2d(256, n_classes, kernel_size=1, stride=1))
        if freeze_bn:
            self._freeze_bn()

    def one_forward(self, input):
        x, low_level_features = self.resnet_features(input)
        x1 = self.aspp1(x)
        x2 = self.aspp2(x)
        x3 = self.aspp3(x)
        x4 = self.aspp4(x)
        x5 = self.global_avg_pool(x)
        x5 = F.upsample(x5, size=x4.size()[2:], mode='bilinear', align_corners=True)

        x = torch.cat((x1, x2, x3, x4, x5), dim=1)

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = F.upsample(x, size=(int(math.ceil(input.size()[-2]/4)),
                                int(math.ceil(input.size()[-1]/4))), mode='bilinear', align_corners=True)

        low_level_features = self.conv2(low_level_features)
        low_level_features = self.bn2(low_level_features)
        low_level_features = self.relu(low_level_features)


        x = torch.cat((x, low_level_features), dim=1)
        x = self.last_conv(x)
        x = F.interpolate(x, size=input.size()[2:], mode='bilinear', align_corners=True)

        return x

    def forward(self, input):
        b = input.size(0)
        x1 = input[:,:,:512,:512]
        x2 = input[:,:,:512,512:]
        x3 = input[:,:,512:,:512]
        x4 = input[:,:,512:,512:]
        x = torch.cat([x1,x2,x3,x4],dim=0)
        x = self.one_forward(x)
        up = torch.cat([x[:b],x[b:2*b]],dim=3)
        down = torch.cat([x[2*b:3*b],x[3*b:]],dim=3)
        out = torch.cat([up,down],dim=2)
        return out

    def _freeze_bn(self):
        for m in self.modules():
            if isinstance(m, BatchNorm2d):
                m.eval()

    def _init_weight(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

In [ ]:
'''
import torch
import torchvision.models as models

resnet101 = models.resnet101(pretrained=True)

torch.save(resnet101.state_dict(), '/personal/预训练模型/resnet101.pth')
'''

In [ ]:
model = DeepLabv3_plus(nInputChannels=1, n_classes=1, os=16, pretrained=True, freeze_bn=False, _print=True)

In [ ]:
import os
import numpy as np
import astropy.io.fits as pyfits
from scipy.ndimage import gaussian_filter
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import matplotlib.pyplot as plt

### Dataset and DataLoader


In [ ]:
class AstroDataset(Dataset):
    def __init__(self, map_paths, cat_paths=None):
        self.map_paths = map_paths
        self.cat_paths = cat_paths
    
    def __len__(self):
        return len(self.map_paths)
    
    def __getitem__(self, idx):
        # Load image with gaussian smoothing
        map_data = gaussian_filter(pyfits.open(self.map_paths[idx])[0].data, sigma=SMOOTHING)
        
        if self.cat_paths is None:
            return torch.FloatTensor(map_data).unsqueeze(0)

        # Load catalog
        cat = pyfits.open(self.cat_paths[idx])[0].data
        target = np.zeros((1024, 1024))

        for y, x in cat[:, 1:3]:
            target[int(y), int(x)] = 1.0

        return torch.FloatTensor(map_data).unsqueeze(0), torch.FloatTensor(target), self.cat_paths[idx]

### Model and Training Functions

In [ ]:
DEVICE = "cuda"

# Function for applicating model results
def detect_objects(model, image_path, confidence_threshold, device=DEVICE):
    model.eval()
    with torch.no_grad():
        # Load and preprocess image
        map_data = gaussian_filter(pyfits.open(image_path)[0].data, sigma=SMOOTHING)
        image = torch.FloatTensor(map_data).unsqueeze(0).unsqueeze(0).to(device)
        
        # Get predictions
        output = model(image)
        output = torch.sigmoid(output)
        predictions = output.cpu().squeeze().numpy()
        
        # Convert to coordinates
        coordinates = []
        for y, x in zip(*np.where(predictions > confidence_threshold)):
            confidence = predictions[y, x]
            coordinates.append((x, y, confidence))
            
        return coordinates

# Visualizing model predictions
def visualize_results(model, image_path, label_path, confidence_threshold, device=DEVICE):
    # Get predictions with confidence scores
    results = detect_objects(model, image_path, confidence_threshold, device)
    results = np.array(results).T if results else np.array([[],[],[]])
    
    Z = pyfits.open(image_path)[0].data
    Z_smooth = gaussian_filter(Z, sigma=SMOOTHING)
    
    labels = np.transpose(pyfits.open(label_path)[0].data)
    
    plt.figure(figsize=(10,10))
    plt.imshow(Z_smooth, vmin=-0.1, vmax=0.2, cmap='binary')
    plt.scatter(labels[2], labels[1], facecolors='none', edgecolors='red', s=100, label="True")
    if len(results[0]) > 0:
        # Color the scatter points based on confidence scores
        plt.scatter(results[0], results[1], facecolors='none', edgecolors='green', s=100, label="Predicted")
    else:
        print("result length 0")
    plt.legend()
    plt.show()


'''
Metric function for calculating PR-AUC.
This exact function will be used for evaluation
'''
def calculate_precision_recall_curve(predictions, labels, verbose = True):

    if verbose:
        print("shape of predictions: ", predictions.shape)

    # Flatten the predictions and get the indices of the sorted predictions
    flat_predictions = predictions.flatten()
    sorted_indices = np.argsort(-flat_predictions)  # Sort in descending order

    precisions = []
    recalls = []

    true_preds = 0
    num_preds = 0
    predicted_labels = 0
    num_labels = sum(len(l) for l in labels)

    labels_within_distance = [[] for _ in range(len(flat_predictions))]

    i = 0
    for image_idx, image_labels in enumerate(labels):
        for y_true, x_true in image_labels:
            for y in range(max(0, int(y_true) - 15), min(1024, int(y_true) + 16)):
                # Calculate the maximum x distance for the current y
                max_x_dist = int((max(0, 15**2 - (y - y_true)**2))**0.5)
                # Calculate the range of x-coordinates
                for x in range(max(0, int(x_true) - max_x_dist), min(1024, int(x_true) + max_x_dist + 1)):
                    coord_idx = image_idx * 1024 * 1024 + y * 1024 + x
                    labels_within_distance[coord_idx].append(i)
            i += 1
    
    label_predicted = [False] * num_labels

    # Iterate over sorted predictions
    for idx in sorted_indices:

        num_preds += 1

        # Determine the image index and the coordinate within the image
        image_idx = idx // (1024 * 1024)
        coord_idx = idx % (1024 * 1024)
        y, x = divmod(coord_idx, 1024)

        if len(labels_within_distance[idx]) > 0:
            true_preds += 1
            for label in labels_within_distance[idx]:
                if label_predicted[label] is False:
                    label_predicted[label] = True
                    predicted_labels += 1

        # Calculate precision and recall
        precision = true_preds / num_preds
        recall = predicted_labels / num_labels

        # Append precision and recall to the lists
        precisions.append(precision)
        recalls.append(recall)

    # Calculate PR-AUC using the trapezoidal rule
    pr_auc = np.trapz(precisions, x=recalls)

    return precisions, recalls, pr_auc

# Evaluate model
def get_pr(model, test_loader, device=DEVICE, verbose=True):
    model.eval()
    outputs_list = []
    labels_list = []
    for images, _, paths in test_loader:
        with torch.no_grad():
            images = images.to(device)
            outputs = torch.sigmoid(model(images)).cpu().numpy().squeeze(1)
            outputs_list.append(outputs)
            cat_data = [np.transpose(pyfits.open(path)[0].data) for path in paths]
            labels = [list(zip(cat[1], cat[2])) for cat in cat_data]
            labels_list += labels
    outputs = np.concat(outputs_list,axis=0)
    return calculate_precision_recall_curve(outputs, labels_list, verbose=verbose)

In [ ]:
# Path to training data *** do not change *** 
DATA_DIR = "/bohr/training-lg02/v1/"

DEVICE = "cuda"
MODEL_SAVE_PATH = "model.pth"

SMOOTHING = 2.0
BATCH_SIZE = 2

data_size = len(os.listdir(os.path.join(DATA_DIR, 'map')))
dataset = AstroDataset([os.path.join(DATA_DIR, f'map/{i}.fits') for i in range(1, 56+1)], [os.path.join(DATA_DIR, f'cat/{i}.fits') for i in range(1, 56+1)])
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

test_set = AstroDataset([os.path.join(DATA_DIR, f'map/{i}.fits') for i in range(57, data_size+1)], [os.path.join(DATA_DIR, f'cat/{i}.fits') for i in range(57, data_size+1)])
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
NUM_EPOCHS = 20
LEARNING_RATE = 0.001

weight = torch.tensor([100.0]).cuda()
criterion = nn.BCEWithLogitsLoss(pos_weight=weight)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

train_losses = []

model.cuda()
for epoch in range(NUM_EPOCHS):
    model.train()
    total_loss = 0
    
    for images, targets, _ in dataloader:
        images = images.to(DEVICE)
        targets = targets.to(DEVICE)
        
        optimizer.zero_grad()
        outputs = model(images).squeeze(1)
        loss = criterion(outputs, targets)
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    train_loss = total_loss/len(dataloader)
    train_losses.append(train_loss)

    model.eval()
    _, _, test_auc = get_pr(model, test_loader, verbose=False)
    _, _, train_auc = get_pr(model, test_loader, verbose=False)

    if (epoch+1)%1 == 0:
        print(f'Epoch [{epoch+1}/{NUM_EPOCHS}], Training Loss: {train_loss:.6f}, Train AUC: {train_auc:.6f}, Test AUC: {test_auc:.6f}')

### Inference

In [ ]:
i = 30
visualize_results(model, dataset.map_paths[i], dataset.cat_paths[i], confidence_threshold=0.5)

In [ ]:
# Calculate PR-AUC
precisions, recalls, pr_auc = get_pr(model, test_loader)
print(f"PR-AUC Score: {pr_auc:.4f}")

# Plot PR curve
plt.figure(figsize=(8, 6))
plt.plot(recalls, precisions)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title(f'Precision-Recall Curve (AUC = {pr_auc:.4f})')
plt.grid(True)
plt.show()